In [1]:
import os
import glob
import pandas as pd
import textgrids
import tqdm
from utils import flac_to_wav

# Create Subset json

## Keep files in subset

In [7]:
import ast


def get_subset_df(json_dir=None):
    if not json_dir:
        json_dir = '/home/knoriy/Documents/laion/split_peoples_speech/flac_train_manifest.jsonl'

    with open(json_dir, 'r') as json_file:
        json_list = list(json_file)

    df = pd.DataFrame.from_dict(json_list)
    df[0] = df[0].apply(lambda x: ast.literal_eval(x))

    for keys in df[0][0].keys():
        df[keys] = [path[0][keys] for path in df.iloc()]

    subset = glob.glob('/home/knoriy/Documents/laion/split_peoples_speech/subset/**/*.flac', recursive=True)
    subset = [os.path.join(*(dir.split(os.path.sep)[7:])) for dir in subset]

    subset_df = df[df['audio_filepath'].isin(subset)].reset_index(drop=True)
    subset_df = subset_df.drop(0, axis=1)

    return subset_df

get_subset_df().to_csv('/home/knoriy/Documents/laion/split_peoples_speech/subset.tsv', sep='\t', header=None, index=False)

# prepare MFA

In [5]:
# Clean old wavs ONLY FOR TESTING
import glob

old_wav_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset/**/*.wav', recursive=True)
for wav_path in old_wav_path:
    os.remove(wav_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset_split/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# Tar loader

# ToDo
- [ ] Download and extract single or batch of files from aws
- [ ] Process for MFA
- [ ] Generate textgrid
- [ ] Split audio and upload to aws s3

In [11]:
import tarfile

In [12]:
filename = '/home/knoriy/Documents/laion/split_peoples_speech/subset_flac.tar'

In [14]:
with tarfile.open(filename,'r') as file_obj:
    file_names = file_obj.getnames()[:10]

    for file_name in file_names:
        os.makedirs(os.path.join('Test', os.path.split(file_name)[0]), exist_ok=True)
        file_obj.extract(file_name, './')


subset
subset/gov_DOT_uscourts_DOT_ca5_DOT_16-41264
subset/gov_DOT_uscourts_DOT_ca5_DOT_16-41264/gov_DOT_uscourts_DOT_ca5_DOT_16-41264_DOT_2017-10-04_DOT_mp3_00051.flac
subset/gov_DOT_uscourts_DOT_ca9_DOT_13-73491
subset/gov_DOT_uscourts_DOT_ca9_DOT_13-73491/gov_DOT_uscourts_DOT_ca9_DOT_13-73491_DOT_2017-11-14_DOT_mp3_00008.flac
subset/snafuinfinityJournal011819-CL19
subset/snafuinfinityJournal011819-CL19/Journal011819-CL19_DOT_mp3_00068.flac
subset/snafuinfinityJournal011819-CL19/Journal011819-CL19_DOT_mp3_00046.flac
subset/Talking_Politics_in_Western_PA_-_Lisa_Anderson_Peters_Township_Board_of_School_Directors
subset/Talking_Politics_in_Western_PA_-_Lisa_Anderson_Peters_Township_Board_of_School_Directors/Talking_Politics_in_Western_PA_-_Lisa_Anderson_Peters_Township_Board_of_School_Directors_DOT_HD_DOT_mp3_00026.flac


In [ ]:
import boto3
import io
import tarfile

class S3File(io.BytesIO):
    def __init__(self, bucket_name, key_name, s3client):
        super().__init__()
        self.bucket_name = bucket_name
        self.key_name = key_name
        self.s3client = s3client
        self.offset = 0

    def close(self):
        return

    def read(self, size):
        print('read: offset = {}, size = {}'.format(self.offset, size))
        start = self.offset
        end = self.offset + size - 1
        try:
            s3_object = self.s3client.get_object(Bucket=self.bucket_name, Key=self.key_name, Range="bytes=%d-%d" % (start, end))
        except:
            return bytearray()
        self.offset = self.offset + size
        result = s3_object['Body'].read()
        return result

    def seek(self, offset, whence=0):
        if whence == 0:
            print('seek: offset {} -> {}'.format(self.offset, offset))
            self.offset = offset

    def tell(self):
        return self.offset

s3file = S3File(bucket_name, file_name, s3client)

# AWS S3

In [ ]:
%%bash

nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-07a8f0d3-6d27-4299-887a-dc12a6d72f8d-c000.tar?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A30%3A22Z/-1/host/444b9c082ceffd10f38bb965679ed9ec12202836831e111dd193fde281062d26 -O - | aws s3 cp - s3://s-laion/peoples_speech/train_clean_pps.tar" &;
nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-4e132642-c01c-4db6-9db0-a1e19193f6f8-c000.json?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A31%3A22Z/-1/host/d7dacf3c31d2e3670d82727636ce234be27a9128df7a80883b84b4a3d8c7f6c0 -O - | aws s3 cp - s3://s-laion/peoples_speech/Manifest.json" &;


In [ ]:
%%bash

nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &



In [ ]:
%%bash

# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &

